# Exercice 1 Réalisation d’un outil pour dessiner des réseaux

Vous devez réaliser un logiciel permettant de dessiner des architectures réseau de façon
simple et élégante. 
Le cahier des charges imposé par le client indique que votre logiciel
doit permettre de représenter :
- des switchs
- des clients (PC ou téléphone)
- des routeurs

Les différents éléments peuvent être présents de multiple fois dans l’interface. 
La création d’un nouvel item pourra se faire par un raccourci clavier ( C client, S switch, R routeur, par exemple ) ou par toolbar / menu, et il doit être possible de déplacer ou d’effacer les items à la souris.

Un clic droit sur un item doit permettre d’en modifier les propriétés :
- Nom
- Icône

Enfin, il doit être possible de dessiner des traits sur le schémas afin de représenter les liens réseau.

In [1]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        # 1) 
        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

# 1) 
root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()

J'ai dans mon premier commit, créer une classe qui permettra d'instancier les objets de l'application car il doivent être présents plusieurs fois en même temps dans la fenêtre.


A la fin de mon programme, je crée une fenêtre qui s'appelle root et à laquelle je créer une instance de la classe "WishPacketTracer".


Et donc le programme va créer un canvas basique de 800x600 qui va s'étendre càd prendre la place dans tout son espace (en l'occurence la fenêtre) et doit s'élargir horizontalement et verticalement à la fois avec le "pack(expand pour l'extension et fill pour l'élargissement)".

<img src="./img/i1.png">

Puis j'ai ajouté à mon code, une barre d'outils dans laquelle, j'ai au préalable chargé mes 3 images (pc,routeur et switch) grâce à PIL et je les redimensionne et je les ai rendues cliquables à l'aide de la métode "Button" dans ma barre d'outils et à l'appui du bouton (donc de l'image), une image va être créée dans mon canvas à l'aide de la méthode "create_image".

In [2]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)
        
        # Barre d'outils
        self.toolbar = tk.Frame(root)
        self.toolbar.pack(side=tk.LEFT, fill=tk.Y)

        # Boutons de la barre d'outils (utilisant des images redimensionnées avec PIL)
        self.image_router = self.load_and_resize_image("./img/Router.png", 50, 50)
        self.button_router = tk.Button(self.toolbar, image=self.image_router, command=self.create_router)
        self.button_router.pack(side=tk.LEFT)

        self.image_switch = self.load_and_resize_image("./img/switch.png", 50, 50)
        self.button_switch = tk.Button(self.toolbar, image=self.image_switch, command=self.create_switch)
        self.button_switch.pack(side=tk.LEFT)

        self.image_pc = self.load_and_resize_image("./img/pc.png", 50, 50)
        self.button_client = tk.Button(self.toolbar, image=self.image_pc, command=self.create_client)
        self.button_client.pack(side=tk.LEFT)

    def create_client(self):
        item = self.canvas.create_image(100, 100, image=self.image_pc, tags="client")
    def create_switch(self):
        item = self.canvas.create_image(200, 200, image=self.image_switch, tags="switch")
    
    def create_router(self):
        item = self.canvas.create_image(300, 300, image=self.image_router, tags="router")
    
   
    def load_and_resize_image(self, filename, width, height):
        image = Image.open(filename)
        image = image.resize((width, height))
        photo = ImageTk.PhotoImage(image)
        return photo

root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()

<img src="./img/i2.png">

Puis, j'ai implémenter la possibilité de pouvoir déplacer les différents items dans mon canvas à l'aide de deux écouteurs d'évènements qui vont se succéder : 
1. "Button-1" Qui va écouter au clic gauche sur mon canvas et va déclencher une fonction "left_click" (clic gauche)


- Et cette fonction, prend en arguments "self" pour l'instance de la classe et "event" (pour le clic càd les coordonnées en abscisses et ordonnées du clic).


- Puis j'affecte à une variable item le résultat de la méthode "find_closest" qui va rechercher l'élément le plus proche des coordonnées du clic (à l'aide des arguments "event.x" pour la coord en abscisse du clic et "event.y" pour la coord en ordonnée du clic).


- Ensuite, une vérification pour savoir s'il y a un élément à l'emplacement du clic et si oui, on défini à une variable instanciée "self.current_item", la 1ère valeur et unique de item.

2. "B1-Motion" Qui va écouter au clic gauche enfoncé sur mon canvas et déclencher une fonction "drag_item" prenant en argument en + de l'instance, "event" pour récupérer les coordonnées du curseur enfoncé.


- Il vérifie d'abord s'il y a une valeur dans "self.current_item" et donc en ayant appuyé avant de pouvoir rester enfoncé, on a déclencher la fonction "left_click" qui a attribué une valeur à "self.current_item".


- Il récupère alors les coordonées actuelles de la souris en abscisses / ordonnées (event.x,event.y) puis va à chacun soustraire les coordonnées actuelles de l'élément sélectionné en abscisses "self.canvas.coords(self.current_item)[0]" et puis en ordonnées "self.canvas.coords(self.current_item)[1]"


- Et la différence des coords actuelles de la souris moins les coords actuelles de l'élément sélectionner nous donne la distance parcourue par la souris avec en abscisses "dx" et en ordonnées "dy".

Ainsi, la méthode "move" permet de déplacer un élément (ici, "self.current_item" donc l'élément sélectionné) de "dx" pixels (une distance horizontale) et de "dy" pixels (une distance verticale).
C'est comme cela, qu'une image peut être "glissée-déposée" (drag-and-drop).

In [3]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        # Barre d'outils
        self.toolbar = tk.Frame(root)
        self.toolbar.pack(side=tk.LEFT, fill=tk.Y)

        # Boutons de la barre d'outils (utilisant des images redimensionnées avec PIL)
        self.image_router = self.load_and_resize_image("./img/Router.png", 50, 50)
        self.button_router = tk.Button(self.toolbar, image=self.image_router, command=self.create_router)
        self.button_router.pack(side=tk.LEFT)

        self.image_switch = self.load_and_resize_image("./img/switch.png", 50, 50)
        self.button_switch = tk.Button(self.toolbar, image=self.image_switch, command=self.create_switch)
        self.button_switch.pack(side=tk.LEFT)

        self.image_pc = self.load_and_resize_image("./img/pc.png", 50, 50)
        self.button_client = tk.Button(self.toolbar, image=self.image_pc, command=self.create_client)
        self.button_client.pack(side=tk.LEFT)

        self.canvas.bind("<Button-1>", self.left_click)
        self.canvas.bind("<B1-Motion>", self.drag_item)

    def create_client(self):
        item = self.canvas.create_image(100, 100, image=self.image_pc, tags="client")

    def create_switch(self):
        item = self.canvas.create_image(200, 200, image=self.image_switch, tags="switch")
    def create_router(self):
        item = self.canvas.create_image(300, 300, image=self.image_router, tags="router")

    def left_click(self, event):
        item = self.canvas.find_closest(event.x, event.y)
        if item:
            self.current_item = item[0]

    def drag_item(self, event):
        if self.current_item:
            dx = event.x - self.canvas.coords(self.current_item)[0]
            dy = event.y - self.canvas.coords(self.current_item)[1]
            self.canvas.move(self.current_item, dx, dy)

    def load_and_resize_image(self, filename, width, height):
        image = Image.open(filename)
        image = image.resize((width, height))
        photo = ImageTk.PhotoImage(image)
        return photo

root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()


Position des éléments de base : 
<img src="./img/i2.png">

Position des élements après déplacement : 
<img src="./img/i3.png">

Il faut également pouvoir créer routeurs / switchs / pc à l'aide d'appuis sur le clavier et c'est pourquoi, j'ai ajouté un écouteur d'évènement "Key" qui va déclencher la fonction "key_pressed".


"key_pressed" est une fonction prenant en argument "event", qui signifie qu'une touche clavier a été tapée puis on y applique la méthode "char" pour retrouver la touche clavier qui a déclenché l'event (ex: on appuie sur la touche "A", "event.char" contiendra "a") et enfin on y applique la méthode "upper" pour s'assurer que la chapine de caractère sera toujours en majuscules et on affecte le résultat à une variable "key".


Ainsi "key" sera une chaine de caractère contenant la touche tapée (ex: on appuie sur la touche "S", key contiendra "S").


Puis on compare "key" à 3 chaine de caractères "C" ou "S" ou "R" et qui vont créer à chacun un pc ou un switch ou un routeur. Si une autre touche a été tapée que ces 3 là alors rien ne se passera.

In [4]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        # Barre d'outils
        self.toolbar = tk.Frame(root)
        self.toolbar.pack(side=tk.LEFT, fill=tk.Y)

        # Boutons de la barre d'outils (utilisant des images redimensionnées avec PIL)
        self.image_router = self.load_and_resize_image("./img/Router.png", 50, 50)
        self.button_router = tk.Button(self.toolbar, image=self.image_router, command=self.create_router)
        self.button_router.pack(side=tk.LEFT)

        self.image_switch = self.load_and_resize_image("./img/switch.png", 50, 50)
        self.button_switch = tk.Button(self.toolbar, image=self.image_switch, command=self.create_switch)
        self.button_switch.pack(side=tk.LEFT)

        self.image_pc = self.load_and_resize_image("./img/pc.png", 50, 50)
        self.button_client = tk.Button(self.toolbar, image=self.image_pc, command=self.create_client)
        self.button_client.pack(side=tk.LEFT)

        # Evènements souris
        self.canvas.bind("<Button-1>", self.left_click)
        self.canvas.bind("<B1-Motion>", self.drag_item)
        # Evènements clavier
        self.root.bind("<Key>", self.key_pressed)

    def create_client(self):
        item = self.canvas.create_image(100, 100, image=self.image_pc, tags="client")

    def create_switch(self):
        item = self.canvas.create_image(200, 200, image=self.image_switch, tags="switch")
    def create_router(self):
        item = self.canvas.create_image(300, 300, image=self.image_router, tags="router")

    def left_click(self, event):
        item = self.canvas.find_closest(event.x, event.y)
        if item:
            self.current_item = item[0]

    def drag_item(self, event):
        if self.current_item:
            dx = event.x - self.canvas.coords(self.current_item)[0]
            dy = event.y - self.canvas.coords(self.current_item)[1]
            self.canvas.move(self.current_item, dx, dy)

    def load_and_resize_image(self, filename, width, height):
        image = Image.open(filename)
        image = image.resize((width, height))
        photo = ImageTk.PhotoImage(image)
        return photo

    def key_pressed(self, event):
        key = event.char.upper()
        if key == "C":
            self.create_client()
        elif key == "S":
            self.create_switch()
        elif key == "R":
            self.create_router()

root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()


Il faut ensuite pouvoir modifier certaines propriétés d'un item sélectionné comme le nom et avoir la possibilité de lui attribuer à nouveau nom.

Du coup, d'abord je crée une liste instanciée "self.items" qui me permettra de stocker les éléments "dessinés" càd les items présents sur mon canva.

Et je crée aussi une variable appellée "self.current_item", qui va stocker l'élément que l'utilisateur a sélectionné.

```Python
...
self.items = []  # Liste pour stocker les éléments dessinés
self.current_item = None  # Stocke l'élément actuellement sélectionné
...
```

J'ajoute aussi à mon programme un menu contextuel "self.context_menu" avec la class "Menu" qui prend 2 argument : 
- le premier est le widget dans lequel il se trouvera (ici root donc la fenetre principale) 
- le second est "tearoff" qui est un argument permettant de gérer le détachement de mon menu contextuel (s'il est à 0 alors mon menu ne se détachera pas du tout et restera fixé dans ma fenetre principale, si par contre il a une autre valeur alors il pourra se détacher càd se trouver dans une fenetre complètement indépendante de ma fenetre principale).

Tearoff à 0 donc pas de détachement du menu 
<img src="./i4.png">

Tearoff à 1 donc détachement du menu (il faut cliquer sur les "-----" pour détacher le menu) : 
<img src="./img/i5.png">

Et dans mon menu contextuel j'utilise la méthode "add_command" qui permet d'ajouter une option dans mon menu qui prendra deux arguments, d'abord "label" pour afficher la description de l'option  mais aussi le lancement de la fonction "edit_proprietes" : 

Sans aucune option dans mon menu (je suis obligé de mettre un tearoff pour pouvoir afficher le petit menu au clic droit sur l'item) : 

<img src="./img/i6.png">

Avec l'option "Modifier les propriétés" (et sans tearoff) : 

<img src="./img/i7.png">

``` Python 
# Menu contextuel
...
self.context_menu = tk.Menu(root, tearoff=0)
self.context_menu.add_command(label="Modifier les propriétés", command=self.edit_proprietes)
...
```

Puis je rajoute dans les évènements de souris l'écouteur d'évènements "Button-3" permettant d'écouter si un clic droit a été fait et si oui qui va lancer la fonction "right_click".

Ensuite j'ai ajouté dans mes fonctions de créations des items (routeurs/switchs/client), l'ajout dans ma liste "items" vu au-dessus d'un dictionnaire correspondant à l'item (en fait je les répertorie à leurs création par ordre de création ex: on crée un routeur, ce sera l'item 1 puis on crée un switch qui sera l'item 2 etc...) contenant ainsi plusieurs infos : 

- item pour le numéro de l'item
- type pour le type d'item
- proprietes qui est un dictionnaire dans le dictionnaire et contiendra "name" pour le nom de l'item (par défaut "Router ou Switch ou Client") et "icon" pour son icône par défaut "C,R,S"

Ex: J'ai crée un routeur puis un switch, si j'affiche la liste "items" : 

<img src="./img/i8.png">
<img src="./img/i9.png">

Ensuite, il faut donc créer la fonction "right_click" qui ne se lance seulement si un clic droit à été fait sur un item prend un argument qui est "event" pour obtenir les coords du clic droit :

```Python
def right_click(self, event):
        item = self.canvas.find_closest(event.x, event.y)
        if item:
            self.current_item = item[0]
            self.context_menu.post(event.x_root, event.y_root)

```
J'affecte à la variable item l'élément le plus proche avec la méthode "find_closest" à l'aide des coordonnées du clic, puis si il y a effectivement un élément dans item alors je défini l'élément sélectionné "self.current_item" à le numéro de l'item (ex :si c'est le premier item créé que j'ai sélectionné alors il aura la valeur 1 ) 

Puis je fais apparaître mon menu contextuel créé au-dessus à l'aide de la méthode "post" qui va afficher au coordonnées du clic droit le menu contextuel.


J'ai par la suite créé ma fonction "edit_proprietes" : 

```Python
def edit_proprietes(self):
        if self.current_item:
            item_type = self.canvas.gettags(self.current_item)[0]
            item = next(x for x in self.items if x["item"] == self.current_item )
            old_name=item["proprietes"]["name"]
            new_name = simpledialog.askstring("Modifier les propriétés", "Nouveau nom:", initialvalue=old_name)
            if new_name:
                item["proprietes"]["name"] = new_name
```
Qui va chercher s'il y a un élément qui a été sélectionné donc s'il y a quelque chose dans "self.current_item" et s'il y a bien un élément sélectionné alors : 

- J'attribue à la variable item_type, le type de l'item (si c'est un routeur, switch ou client) grâce à la méthode "gettags" qui va prendre le tag de l'image du canva, puisqu'à la création de l'image sur le canva, on lui a attribué un tag.

- Je parcours ensuite ma liste "items" contenant les items créés sur le canva et grâce à la fonction "next" en compréhension de liste qui parcours et ,s'il trouve arrête de parcourir ma liste, sous une condition marquée par le "if" qui précise qu'il faut que le numéro de l'élément sélectionné corresponde bien à l'élément sélectionné (ex: si on créer 5 routeurs et que l'on cherche le 4e routeur alors il faut bien s'assurer que le routeur que l'on sélectionne soit le 4e routeur créé).

J'ai crée ci-dessous 4 routeurs que j'ai placé par ordre de création dans le sens des aiguilles d'une montre (le 1er routeur étant en haut à gauche) et dont j'ai sélectionné le 3e routeur

<img src="./img/i10.png">

Ma fonction (en affichant "item" dans la fonction "edit_proprietes") me donne bien l'item 3 correspondant au routeur 3 : 

<img src="./img/i11.png">

Puis je récupère l'ancien nom du routeur (par défaut est "Router")

Et à l'aide du module "simpledialog" de tkinter et sa méthode "askstring", je crée une fenetre dans laquelle, l'utilisateur peut attribuer un nouveau nom (initialvalue permet d'inscrire dans le champ à modifier, l'ancien nom de l'élément).

j'affecte la valeur du champ donc le nouveau nom inscrit par l'utilisateur à une variable new_name et enfin je vérifie s'il un nouveau nom a été attribué et si oui alors je change dans le dictionnaire de l'élément sélectionné, le nom de l'élément.

J'ai au préalable changé le nom d'un routeur pour vérifier si ma fenetre de nouveau nom reprenait bien l'ancien nom de l'élément pour l'afficher (l'ancien nom étant "AncienNom") : 

<img src="./img/i12.png">

Puis je change le nom de cet élément en lui donnant "NouveauNom" comme nom : 

<img src="./img/i13.png">

Et si je revérifie en allant dnas la fenetre nouveau nom, je vois bien que le nom a été changé : 

<img src="./img/i14.png">

Et donc le code total à ce point : 


In [5]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        self.items = []  # Liste pour stocker les éléments dessinés
        self.current_item = None  # Stocke l'élément actuellement sélectionné


        # Barre d'outils
        self.toolbar = tk.Frame(root)
        self.toolbar.pack(side=tk.LEFT, fill=tk.Y)

        # Boutons de la barre d'outils (utilisant des images redimensionnées avec PIL)
        self.image_router = self.load_and_resize_image("./img/Router.png", 50, 50)
        self.button_router = tk.Button(self.toolbar, image=self.image_router, command=self.create_router)
        self.button_router.pack(side=tk.LEFT)

        self.image_switch = self.load_and_resize_image("./img/switch.png", 50, 50)
        self.button_switch = tk.Button(self.toolbar, image=self.image_switch, command=self.create_switch)
        self.button_switch.pack(side=tk.LEFT)

        self.image_pc = self.load_and_resize_image("./img/pc.png", 50, 50)
        self.button_client = tk.Button(self.toolbar, image=self.image_pc, command=self.create_client)
        self.button_client.pack(side=tk.LEFT)

        # Menu contextuel
        self.context_menu = tk.Menu(root, tearoff=0)
        self.context_menu.add_command(label="Modifier les propriétés", command=self.edit_proprietes)


        # Evènements souris
        self.canvas.bind("<Button-1>", self.left_click)
        self.canvas.bind("<Button-3>", self.right_click)
        self.canvas.bind("<B1-Motion>", self.drag_item)
        # Evènements clavier
        self.root.bind("<Key>", self.key_pressed)

        

    def create_client(self):
        item = self.canvas.create_image(100, 100, image=self.image_pc, tags="client")
        self.items.append({"item": item, "type": "client", "proprietes": {"name": "Client", "icon": "C"}})

    def create_switch(self):
        item = self.canvas.create_image(200, 200, image=self.image_switch, tags="switch")
        self.items.append({"item": item, "type": "switch", "proprietes": {"name": "Switch", "icon": "S"}})

    def create_router(self):
        item = self.canvas.create_image(300, 300, image=self.image_router, tags="router")
        self.items.append({"item": item, "type": "router", "proprietes": {"name": "Router", "icon": "R"}})
    
        

    def left_click(self, event):
        item = self.canvas.find_closest(event.x, event.y)
        if item:
            self.current_item = item[0]

    def right_click(self, event):
        item = self.canvas.find_closest(event.x, event.y)
        if item:
            self.current_item = item[0]
            self.context_menu.post(event.x_root, event.y_root)

    def edit_proprietes(self):
        if self.current_item:
            item_type = self.canvas.gettags(self.current_item)[0]
            item = next(x for x in self.items if x["item"] == self.current_item )
            old_name=item["proprietes"]["name"]
            new_name = simpledialog.askstring("Modifier les propriétés", "Nouveau nom:", initialvalue=old_name)
            if new_name:
                item["proprietes"]["name"] = new_name

    def drag_item(self, event):
        if self.current_item:
            dx = event.x - self.canvas.coords(self.current_item)[0]
            dy = event.y - self.canvas.coords(self.current_item)[1]
            self.canvas.move(self.current_item, dx, dy)

    def load_and_resize_image(self, filename, width, height):
        image = Image.open(filename)
        image = image.resize((width, height))
        photo = ImageTk.PhotoImage(image)
        return photo

    def key_pressed(self, event):
        key = event.char.upper()
        if key == "C":
            self.create_client()
        elif key == "S":
            self.create_switch()
        elif key == "R":
            self.create_router()

root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()


Ensuite, il faut pouvoir supprimer les items à l'aide du clic droit, pour ce faire, j'ai ajouter une nouvelle option lors du clic droit d'un item qui est "Supprimer" et va lancer la fonction "delete_item" :

```Python
self.context_menu.add_command(label="Supprimer", command=self.delete_item)

...

def delete_item(self):
        if self.current_item:
            item = next(x for x in self.items if x["item"] == self.current_item)
            self.items.remove(item)
            self.canvas.delete(self.current_item)
...
```

La fonction delete_item va d'abord vérifier s'il un item a été sélectionné donc en regardant s'il y a une valeur dans "self.current_item" et puis il va encore parcourir la liste des items pour faire correspondre le numéro de l'item sélectionné avec l'item sélectionné (ex: si on veut supprimer le 2e routeur crée, il faut vérifier que le routeur sur lequel on a fait clic droit porte bien le numéro 2).

Et une fois trouvé, il va l'affecter à une variable "item" qui va nous servir à retirer les infos de l'élément sélectionné (si on supprime le switch 2 il faut que les infos du switch 2 seulement soient retirées de la liste des items).

On retire également l'image de l'élément sur le canva à l'aide de la méthode delete.

Ex: J'ai créé 3 routeurs et j'affiche la liste des items à chaque clic droit ;
<img src="./img/i15.png">

dans la liste des items on a donc 3 dictionnaires avec items "1,2 et 3" ; 

<img src="./img/i16.png">

Puis je suppr le second item : 

<img src="./img/i17.png">

Et j'ai bien le dico du second routeurs qui a été retiré de la liste des items (passant de l'item 1 à l'item 3) : 

<img src="./img/i18.png">

Et j'ai bien seulement 2 routeurs sur mon canva :

<img src="./img/i19.png">

Donc le code à ce point est : 


In [6]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        self.items = []  # Liste pour stocker les éléments dessinés
        self.current_item = None  # Stocke l'élément actuellement sélectionné


        # Barre d'outils
        self.toolbar = tk.Frame(root)
        self.toolbar.pack(side=tk.LEFT, fill=tk.Y)

        # Boutons de la barre d'outils (utilisant des images redimensionnées avec PIL)
        self.image_router = self.load_and_resize_image("./img/Router.png", 50, 50)
        self.button_router = tk.Button(self.toolbar, image=self.image_router, command=self.create_router)
        self.button_router.pack(side=tk.LEFT)

        self.image_switch = self.load_and_resize_image("./img/switch.png", 50, 50)
        self.button_switch = tk.Button(self.toolbar, image=self.image_switch, command=self.create_switch)
        self.button_switch.pack(side=tk.LEFT)

        self.image_pc = self.load_and_resize_image("./img/pc.png", 50, 50)
        self.button_client = tk.Button(self.toolbar, image=self.image_pc, command=self.create_client)
        self.button_client.pack(side=tk.LEFT)

        # Menu contextuel
        self.context_menu = tk.Menu(root, tearoff=0)
        self.context_menu.add_command(label="Modifier les propriétés", command=self.edit_proprietes)
        self.context_menu.add_command(label="Supprimer", command=self.delete_item)


        # Evènements souris
        self.canvas.bind("<Button-1>", self.left_click)
        self.canvas.bind("<Button-3>", self.right_click)
        self.canvas.bind("<B1-Motion>", self.drag_item)
        # Evènements clavier
        self.root.bind("<Key>", self.key_pressed)

        

    def create_client(self):
        item = self.canvas.create_image(100, 100, image=self.image_pc, tags="client")
        self.items.append({"item": item, "type": "client", "proprietes": {"name": "Client", "icon": "C"}})

    def create_switch(self):
        item = self.canvas.create_image(200, 200, image=self.image_switch, tags="switch")
        self.items.append({"item": item, "type": "switch", "proprietes": {"name": "Switch", "icon": "S"}})

    def create_router(self):
        item = self.canvas.create_image(300, 300, image=self.image_router, tags="router")
        self.items.append({"item": item, "type": "router", "proprietes": {"name": "Router", "icon": "R"}})
    
        

    def left_click(self, event):
        item = self.canvas.find_closest(event.x, event.y)
        if item:
            self.current_item = item[0]

    def right_click(self, event):
        item = self.canvas.find_closest(event.x, event.y)
        if item:
            self.current_item = item[0]
            self.context_menu.post(event.x_root, event.y_root)

    def edit_proprietes(self):
        if self.current_item:
            item_type = self.canvas.gettags(self.current_item)[0]
            # print(self.items)
            item = next(x for x in self.items if x["item"] == self.current_item )
            old_name=item["proprietes"]["name"]
            new_name = simpledialog.askstring("Modifier les propriétés", "Nouveau nom:", initialvalue=old_name)
            if new_name:
                item["proprietes"]["name"] = new_name

    def drag_item(self, event):
        if self.current_item:
            dx = event.x - self.canvas.coords(self.current_item)[0]
            dy = event.y - self.canvas.coords(self.current_item)[1]
            self.canvas.move(self.current_item, dx, dy)

    def load_and_resize_image(self, filename, width, height):
        image = Image.open(filename)
        image = image.resize((width, height))
        photo = ImageTk.PhotoImage(image)
        return photo

    def key_pressed(self, event):
        key = event.char.upper()
        if key == "C":
            self.create_client()
        elif key == "S":
            self.create_switch()
        elif key == "R":
            self.create_router()

    def delete_item(self):
        if self.current_item:
            item = next(x for x in self.items if x["item"] == self.current_item)
            self.items.remove(item)
            self.canvas.delete(self.current_item)

root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()


# Exercice 2 Des traits bien droits !
Faites en sorte que les lignes ne puissent être que horizontales ou verticales lorsque
l’utilisateur maintient la touche CTRL enfoncée.


# Exercice 3 Un client exigeant ...
Le client, satisfait par votre travail précédant, demande à ce que vous alliez plus loin. Il
souhaite maintenant que les items aient des ports sur lesquels votre souris sera aimantée
lorsque vous dessinez des traits. Le nombre de ports des routeurs et des switchs sera
paramétrable de 1 à 4. Les clients, eux, auront toujours un seul port